In [26]:
import pandas as pd
import re
import numpy as np
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
import os
from datetime import datetime, timedelta
import smtplib
from IPython.display import display, HTML
import subprocess


# === CONFIG ===
os.chdir(r'C:\Users\wws5213\CFB Email Scores Scraping Project/cfb_boxscores')
GIT_PATH = r"C:\Users\wws5213\AppData\Local\Programs\Git\cmd\git.exe"
REPO_DIR = r"C:\Users\wws5213\CFB Email Scores Scraping Project\cfb_boxscores"
NEWSLETTER_TYPES = ['Big Ten']  # Or just ['ALL'] for everything
# or, to send everything: # NEWSLETTER_TYPES = ['ALL']  # Options: 'Big Ten', 'Big 12', 'ACC', 'SEC', 'Top 25', 'Ranked GO5'
VALID_NEWSLETTERS = ['Big Ten', 'Big 12', 'ACC', 'SEC', 'Top 25', 'Ranked GO5']
DATE_OVERRIDE = '2024-11-17'         # 'YYYY-MM-DD' for testing, else None for yesterday
RECIPIENTS = ["will.semmer@gmail.com"]
SENDER = "will.semmer@gmail.com"
SMTP_USER = "will.semmer@gmail.com"
SMTP_PASSWORD = "lzdf ngyw lvqg trlv"  # <-- put yours here
SMTP_SERVER = "smtp.gmail.com"
SMTP_PORT = 465
NEWSLETTER_AUTHOR = "Will Semmer"

MASTER_DATA_PATH = 'cfb_allgames_2024.csv'
TOP25_RANKINGS_PATH = 'espn_top25_weeks1-16_2024.csv'
TEAM_LOGO_LOOKUP_PATH = 'download.csv'
LOGO_BASE = 'logos'


In [27]:
def minify_html(html):
    # Collapse runs of whitespace to a single space
    html = re.sub(r'\s{2,}', ' ', html)
    # Remove whitespace at the start and end
    return html.strip()

In [28]:
def publish_html_file(filename, html_code, commit_msg=None):
    # Write the HTML file
    file_path = os.path.join(REPO_DIR, filename)
    with open(file_path, "w", encoding="utf-8") as f:
        f.write(html_code)
    print("Saved file:", file_path)
    # Git add/commit/push
    subprocess.run([GIT_PATH, "add", filename], cwd=REPO_DIR, check=True)
    if not commit_msg:
        commit_msg = f"Add/update game HTML: {filename}"
    subprocess.run([GIT_PATH, "commit", "-m", commit_msg], cwd=REPO_DIR, check=True)
    subprocess.run([GIT_PATH, "push"], cwd=REPO_DIR, check=True)
    # Construct URL
    github_url = f"https://semmertime.github.io/cfb_boxscores/{filename}"
    return github_url


In [29]:
def get_newsletter_date(date_override=None):
    # Always return the date to FILTER ON (yesterday if live, else the day before override)
    if date_override:
        # date_override should be 'today', so subtract one day
        dt = pd.to_datetime(date_override)
        return (dt - pd.Timedelta(days=1)).strftime('%Y-%m-%d')
    else:
        return (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')

# --- Conference definitions for filtering ---
GO5_CONFERENCES = [
    "American Athletic",
    "Conference USA",
    "Mid-American",
    "Mountain West",
    "Sun Belt",
    "Pac-12"
]
GO5_INDEPENDENTS = ["UConn", "UMass"]  # Only these count as "GO5" for newsletter purposes

def filter_games(df, newsletter_type):
    if newsletter_type in ['Big Ten', 'Big 12', 'SEC']:
        # Standard power conference filter
        return df[(df['Home_Conference'] == newsletter_type) | (df['Away_Conference'] == newsletter_type)].copy()
    elif newsletter_type == 'ACC':
        # ACC filter, but include games involving Notre Dame (as home or away)
        return df[
            (df['Home_Conference'] == 'ACC') | (df['Away_Conference'] == 'ACC') |
            (df['Home_Team'] == "Notre Dame") | (df['Away_Team'] == "Notre Dame")
        ].copy()
    elif newsletter_type == 'Top 25':
        return df[(df['Home_Rank'].notnull() & (df['Home_Rank'] <= 25)) |
                  (df['Away_Rank'].notnull() & (df['Away_Rank'] <= 25))].copy()
    elif newsletter_type == 'Ranked GO5':
        mask = (
            # GO5 conferences, home or away, must be ranked
            ((df['Home_Conference'].isin(GO5_CONFERENCES)) & (df['Home_Rank'].notnull()) & (df['Home_Rank'] <= 25)) |
            ((df['Away_Conference'].isin(GO5_CONFERENCES)) & (df['Away_Rank'].notnull()) & (df['Away_Rank'] <= 25)) |
            # Independents UConn/UMass, must be ranked
            ((df['Home_Conference'] == 'FBS Independents') & (df['Home_Team'].isin(GO5_INDEPENDENTS)) & (df['Home_Rank'].notnull()) & (df['Home_Rank'] <= 25)) |
            ((df['Away_Conference'] == 'FBS Independents') & (df['Away_Team'].isin(GO5_INDEPENDENTS)) & (df['Away_Rank'].notnull()) & (df['Away_Rank'] <= 25))
        )
        return df[mask].copy()
    else:
        raise ValueError(f"Unknown newsletter type: {newsletter_type}")


In [30]:
TEAM_NAME_FIXES = {
    "NC State": "NC State",
    "NCSU NC State": "NC State",
    "N.C. State": "NC State",
    "North Carolina State": "NC State",
    "State": "NC State",  # If you want ALL "State" to become "NC State" (could be risky)
    # Add more special cases as needed
}


def clean_team_name(team):
    team = str(team).strip()
    # FIRST: Handle exact matches in the fixes dictionary
    if team in TEAM_NAME_FIXES:
        return TEAM_NAME_FIXES[team]
    # Remove codes at the start
    team = re.sub(r"^[A-Z0-9&\.\'\-\*]{2,5}\s+", "", team)
    # Remove (xx) votes at end
    team = re.sub(r"\s*\(\d+\)", "", team)
    # Remove any leading/trailing asterisks
    team = re.sub(r"^\*+", "", team)
    team = re.sub(r"\*+$", "", team)
    team = team.strip()
    # LAST: If after all this, it's just "State", map it to "NC State"
    if team == "State":
        return "NC State"
    return team




def attach_rankings(df2_clean, all_rankings):
    df = df2_clean.copy()
    ranks = all_rankings.copy()

    # Make Week columns string for matching, and create Week_for_merge for week 0 logic
    df['Week'] = df['Week'].astype(str)
    ranks['Week'] = ranks['Week'].astype(str)
    df['Week_for_merge'] = df['Week'].replace({'0': '1'})
    ranks['Week_for_merge'] = ranks['Week']

    # Clean team names
    df['Home_Team_Clean'] = df['Home_Team'].apply(clean_team_name)
    df['Away_Team_Clean'] = df['Away_Team'].apply(clean_team_name)
    ranks['Team_Clean'] = ranks['Team'].apply(clean_team_name)

    # Remove any prior rank columns
    for col in ['Home_Rank', 'Away_Rank']:
        df = df.drop(columns=[c for c in df.columns if c.startswith(col)], errors='ignore')

    # Merge home ranks
    df = df.merge(
        ranks[['Week_for_merge', 'Team_Clean', 'RK']].rename(
            columns={'Team_Clean': 'Home_Team_Clean', 'RK': 'Home_Rank'}
        ),
        on=['Week_for_merge', 'Home_Team_Clean'],
        how='left'
    )

    # Merge away ranks
    df = df.merge(
        ranks[['Week_for_merge', 'Team_Clean', 'RK']].rename(
            columns={'Team_Clean': 'Away_Team_Clean', 'RK': 'Away_Rank'}
        ),
        on=['Week_for_merge', 'Away_Team_Clean'],
        how='left'
    )

    # Deduplicate columns, keeping only the final Home_Rank and Away_Rank
    for base in ['Home_Rank', 'Away_Rank']:
        cols = [c for c in df.columns if c.startswith(base)]
        if cols:
            df[base] = df[cols[-1]]
            for col in cols:
                if col != base:
                    df = df.drop(columns=[col])
    return df







def get_full_top25_standings_table(newsletter_week, newsletter_date, merged_df, rankings_df, Team_Colors):
    """
    newsletter_week: int or str, the poll week to display (e.g. '2' for Week 2 poll)
    newsletter_date: str or pd.Timestamp, the cutoff date for records (e.g. Sunday night of that week)
    merged_df: your merged games DataFrame, with Home_Rank/Away_Rank columns
    rankings_df: the original rankings file (with Week, Team, RK, etc.)
    Team_Colors: your color dictionary
    """
    # Get Top 25 teams for the poll week
    week_str = str(newsletter_week)
    poll = rankings_df[rankings_df['Week'] == int(week_str)].copy()
    poll['Team_Clean'] = poll['Team'].apply(clean_team_name)
    poll = poll[['RK', 'Team', 'Team_Clean']].sort_values('RK')

    # For each Top 25 team, get latest record/conf record and next opp as of newsletter_date
    standings = []
    date = pd.to_datetime(newsletter_date)

    for _, row in poll.iterrows():
        team = row['Team_Clean']
        rank = int(row['RK'])
        # Find last game played on/before date
        mask_home = (merged_df['Home_Team_Clean'] == team)
        mask_away = (merged_df['Away_Team_Clean'] == team)
        games_team = merged_df[(mask_home | mask_away)]
        games_team = games_team[pd.to_datetime(games_team['Start_Date']) <= date]
        if not games_team.empty:
            last_game = games_team.sort_values('Start_Date', ascending=False).iloc[0]
            if last_game['Home_Team_Clean'] == team:
                overall_record = last_game['Home_Record']
                conf_record = last_game['Home_Conf_Record']
            else:
                overall_record = last_game['Away_Record']
                conf_record = last_game['Away_Conf_Record']
        else:
            overall_record, conf_record = "", ""

        # Find next game after date
        future_games = merged_df[(mask_home | mask_away) & (pd.to_datetime(merged_df['Start_Date']) > date)].sort_values('Start_Date')
        if not future_games.empty:
            next_game = future_games.iloc[0]
            if next_game['Home_Team_Clean'] == team:
                opp = next_game['Away_Team_Clean']
            else:
                opp = next_game['Home_Team_Clean']
            # Get opponent's rank for that week, if any
            opp_rank = ""
            if next_game['Home_Team_Clean'] == team:
                opp_rank = next_game['Away_Rank']
            else:
                opp_rank = next_game['Home_Rank']
            venue = "" if next_game['Home_Team_Clean'] == team else "@"
            next_game_str = f"{venue}{opp}"
            next_game_date = next_game['Start_Date']
        else:
            next_game_str = ""
            opp_rank = ""
            next_game_date = ""
        standings.append({
            "Team": team,
            "Rank": rank,
            "Overall_Record": overall_record,
            "Conf_Record": conf_record,
            "Next_Game": next_game_str,
            "Next_Game_Rank": int(opp_rank) if pd.notnull(opp_rank) and opp_rank != "" else "",
            "Next_Game_Date": next_game_date
        })

    # Now use your existing HTML rendering code!
    # Pass 'standings' to your build_standings_table logic, or tweak as needed.
    return standings

In [31]:
Team_Colors = {
    "Air Force": "#003087",
    "Akron": "#041E42",
    "Alabama": "#9E1B32",
    "Appalachian State": "#222222",
    "Arizona": "#CC0033",
    "Arizona State": "#942139",
    "Arkansas": "#9D2235",
    "Arkansas State": "#CC092F",
    "Army": "#9E7E38",
    "Auburn": "#0C2340",
    "Ball State": "#BA0C2F",
    "Baylor": "#154734",
    "Boise State": "#0033A0",
    "Boston College": "#8A100B",
    "Bowling Green": "#402419",
    "Buffalo": "#005BBB",
    "BYU": "#002E5D",
    "California": "#003262",
    "Central Michigan": "#6A0032",
    "Charlotte": "#046A38",
    "Cincinnati": "#D50A0A",
    "Clemson": "#F56600",
    "Coastal Carolina": "#008E97",
    "Colorado": "#CFB87C",
    "Colorado State": "#215732",
    "Connecticut": "#003069",
    "Duke": "#00539B",
    "East Carolina": "#4B1869",
    "Eastern Michigan": "#006633",
    "FIU": "#081E3F",
    "Florida": "#FA4616",
    "Florida Atlantic": "#003366",
    "Florida State": "#782F40",
    "Fresno State": "#C41230",
    "Georgia": "#BA0C2F",
    "Georgia Southern": "#041E42",
    "Georgia State": "#0039A6",
    "Georgia Tech": "#B3A369",
    "Hawaii": "#024731",
    "Houston": "#C8102E",
    "Illinois": "#E84A27",
    "Indiana": "#990000",
    "Iowa": "#FFCD00",
    "Iowa State": "#C8102E",
    "Jacksonville State": "#D11F3E",
    "James Madison": "#450084",
    "Kansas": "#0051BA",
    "Kansas State": "#512888",
    "Kent State": "#003776",
    "Kentucky": "#0033A0",
    "Liberty": "#A41E34",
    "Louisiana": "#CE181E",
    "Louisiana-Monroe": "#882D17",
    "Louisiana Tech": "#003087",
    "Louisville": "#AD0000",
    "LSU": "#461D7C",
    "Marshall": "#00B140",
    "Maryland": "#E03A3E",
    "Memphis": "#0046AD",
    "Miami (FL)": "#F47321",
    "Miami (OH)": "#B31B1B",
    "Michigan": "#00274C",
    "Michigan State": "#18453B",
    "Middle Tennessee": "#0066CC",
    "Minnesota": "#7A0019",
    "Mississippi State": "#660000",
    "Missouri": "#F1B82D",
    "Navy": "#00205B",
    "NC State": "#CC0000",
    "Nebraska": "#E41C38",
    "Nevada": "#003366",
    "New Mexico": "#D50032",
    "New Mexico State": "#861F41",
    "North Carolina": "#4B9CD3",
    "North Texas": "#00853E",
    "Northern Illinois": "#A6192E",
    "Northwestern": "#4E2A84",
    "Notre Dame": "#C99700",
    "Ohio": "#00703C",
    "Ohio State": "#BB0000",
    "Oklahoma": "#841617",
    "Oklahoma State": "#FF7300",
    "Old Dominion": "#003057",
    "Ole Miss": "#006BA6",
    "Oregon": "#154733",
    "Oregon State": "#DC4405",
    "Penn State": "#041E42",
    "Pittsburgh": "#003594",
    "Purdue": "#CEB888",
    "Rice": "#00205B",
    "Rutgers": "#CC0033",
    "San Diego State": "#A6192E",
    "San Jose State": "#0055A2",
    "SMU": "#C0043F",
    "South Alabama": "#00205B",
    "South Carolina": "#73000A",
    "South Florida": "#006747",
    "Southern Miss": "#FFB612",
    "Stanford": "#8C1515",
    "Syracuse": "#F76900",
    "TCU": "#4D1979",
    "Temple": "#9D2235",
    "Tennessee": "#FF8200",
    "Texas": "#BF5700",
    "Texas A&M": "#500000",
    "Texas State": "#501214",
    "Texas Tech": "#CC0000",
    "Toledo": "#15397F",
    "Troy": "#7C878E",
    "Tulane": "#006747",
    "Tulsa": "#0066CC",
    "UAB": "#007A33",
    "UCF": "#BBA14F",
    "UCLA": "#2774AE",
    "UConn": "#003069",
    "UMass": "#881C1C",
    "UNLV": "#D8262C",
    "USC": "#990000",
    "UTEP": "#041E42",
    "UTSA": "#0C2340",
    "Utah": "#CC0000",
    "Utah State": "#0F2439",
    "Vanderbilt": "#866D4B",
    "Virginia": "#232D4B",
    "Virginia Tech": "#861F41",
    "Wake Forest": "#9E7E38",
    "Washington": "#4B2E83",
    "Washington State": "#981E32",
    "West Virginia": "#002855",
    "Western Kentucky": "#D40A2F",
    "Western Michigan": "#8B5B29",
    "Wisconsin": "#C5050C",
    "Wyoming": "#FFC425",
    "Abilene Christian": "#4E2A84",
    "Alabama A&M": "#88131A",
    "Alabama State": "#000000",
    "Albany": "#461D7C",
    "Alcorn State": "#582C83",
    "American International": "#FDB913",
    "Arkansas-Pine Bluff": "#231F20",
    "Austin Peay": "#CC092F",
    "Bethune-Cookman": "#5E2129",
    "Big South": "#1B365D",
    "Brown": "#4E3629",
    "Bryant": "#A89968",
    "Bucknell": "#FF5F00",
    "Butler": "#003087",
    "Cal Poly": "#0B6B3A",
    "Campbell": "#F47A20",
    "Central Arkansas": "#4F2C85",
    "Central Connecticut": "#0033A0",
    "Charleston Southern": "#002A5C",
    "Chattanooga": "#0C2340",
    "Clark Atlanta": "#B31B1B",
    "Colgate": "#862633",
    "College of Charleston": "#6C1D45",
    "Columbia": "#0088CE",
    "Cornell": "#B31B1B",
    "Dartmouth": "#00693E",
    "Davidson": "#D52B1E",
    "Dayton": "#CF0A2C",
    "Delaware": "#00539B",
    "Delaware State": "#E51B24",
    "Drake": "#0061AA",
    "Duquesne": "#041E42",
    "East Tennessee State": "#041E42",
    "Eastern Illinois": "#0033A0",
    "Eastern Kentucky": "#800000",
    "Eastern Washington": "#E30B17",
    "Elon": "#7A0019",
    "Florida A&M": "#00563F",
    "Fordham": "#890024",
    "Furman": "#582C83",
    "Gardner-Webb": "#BE1E2D",
    "Georgetown": "#818181",
    "Grambling State": "#FFD100",
    "Harvard": "#A51C30",
    "Holy Cross": "#512698",
    "Howard": "#00205B",
    "Idaho": "#B3995D",
    "Idaho State": "#E87722",
    "Illinois State": "#D31145",
    "Incarnate Word": "#E4002B",
    "Indiana State": "#0072CE",
    "Jackson State": "#002147",
    "Jacksonville State": "#D11F3E",
    "James Madison": "#450084",
    "Kennesaw State": "#FDB913",
    "Lafayette": "#781214",
    "Lamar": "#A6192E",
    "Lehigh": "#6B4C27",
    "Liberty": "#A41E34",
    "Long Island": "#7AC9E3",
    "Maine": "#003052",
    "Marist": "#C8102E",
    "Marshall": "#00B140",
    "McNeese State": "#00205B",
    "Mercer": "#FF8200",
    "Merrimack": "#142952",
    "Mississippi Valley State": "#007749",
    "Missouri State": "#4E2A84",
    "Monmouth": "#013E7C",
    "Montana": "#660000",
    "Montana State": "#003366",
    "Morehead State": "#0071BC",
    "Morgan State": "#0077C8",
    "Murray State": "#0C2340",
    "New Hampshire": "#003366",
    "Nicholls State": "#C8102E",
    "Norfolk State": "#18453B",
    "North Alabama": "#4F2C85",
    "North Carolina A&T": "#0033A0",
    "North Carolina Central": "#990000",
    "North Dakota": "#009A44",
    "North Dakota State": "#115740",
    "Northern Arizona": "#003366",
    "Northern Colorado": "#002F6C",
    "Northern Iowa": "#4F2C85",
    "Northwestern State": "#4F2C85",
    "Penn": "#990000",
    "Portland State": "#154734",
    "Prairie View A&M": "#5C068C",
    "Presbyterian": "#00205B",
    "Princeton": "#FF8F00",
    "Rhode Island": "#0098D8",
    "Richmond": "#BA0C2F",
    "Robert Morris": "#041E42",
    "Sacramento State": "#043927",
    "Sacred Heart": "#C8102E",
    "Sam Houston State": "#FF8200",
    "Samford": "#002F6C",
    "San Diego": "#003057",
    "Savannah State": "#F66733",
    "Southeast Missouri State": "#D50032",
    "Southeastern Louisiana": "#006A4D",
    "Southern": "#003087",
    "Southern Illinois": "#542E91",
    "South Carolina State": "#4F2C85",
    "South Dakota": "#A6192E",
    "South Dakota State": "#0033A0",
    "Southern Utah": "#B30C2B",
    "Stephen F. Austin": "#4F2C85",
    "Stetson": "#00563F",
    "Stonehill": "#A2AAAD",
    "Stony Brook": "#C60C30",
    "Tarleton State": "#582C83",
    "Tennessee State": "#00205B",
    "Tennessee Tech": "#7C878E",
    "Texas Southern": "#660000",
    "The Citadel": "#A7C6ED",
    "Towson": "#FFB612",
    "Troy": "#7C878E",
    "UC Davis": "#002855",
    "Utah Tech": "#D22630",
    "Valparaiso": "#866D4B",
    "Villanova": "#003366",
    "VMI": "#A6192E",
    "Wagner": "#006B54",
    "Wake Forest": "#9E7E38",
    "Weber State": "#4F2C85",
    "Western Carolina": "#633194",
    "Western Illinois": "#6F263D",
    "William & Mary": "#115740",
    "Wofford": "#A89968",
    "Yale": "#0F4D92"
}

In [32]:
# Load data
df_all = pd.read_csv(MASTER_DATA_PATH)
df_all['Start_Date'] = pd.to_datetime(df_all['Start_Date']).dt.strftime('%Y-%m-%d')
df_lookup = pd.read_csv(TEAM_LOGO_LOOKUP_PATH)
team_to_code = dict(zip(df_lookup['School'], df_lookup['Id'].astype(str)))
rankings_df = pd.read_csv(TOP25_RANKINGS_PATH)
rankings_df['Team_Clean'] = rankings_df['Team'].apply(clean_team_name)
df_all['Home_Team_Clean'] = df_all['Home_Team'].apply(clean_team_name)
df_all['Away_Team_Clean'] = df_all['Away_Team'].apply(clean_team_name)
df_all = attach_rankings(df_all, rankings_df)

# Date
yesterday_str = get_newsletter_date(DATE_OVERRIDE)

for NEWSLETTER_TYPE in VALID_NEWSLETTERS:
    newsletter_games = filter_games(df_all, NEWSLETTER_TYPE)

    if NEWSLETTER_TYPE == "Top 25":
        # ==== THIS IS YOUR AUTOMATIC, NEVER-HARD-CODED LOGIC ====
        games_dates = pd.to_datetime(df_all['Start_Date'])
        yesterday = pd.to_datetime(yesterday_str)
        games_before = df_all[games_dates <= yesterday]
        week_for_poll = pd.to_numeric(games_before['Week'], errors='coerce').max()
        available_poll_weeks = sorted(rankings_df['Week'].unique())
        if pd.isnull(week_for_poll) or week_for_poll < min(available_poll_weeks):
            poll_week = min(available_poll_weeks)
        elif week_for_poll > max(available_poll_weeks):
            poll_week = max(available_poll_weeks)
        else:
            poll_week = max(w for w in available_poll_weeks if w <= week_for_poll)

        print("DEBUG: Top 25 poll_week being used:", poll_week)

        standings = get_full_top25_standings_table(
            newsletter_week=poll_week,
            newsletter_date=yesterday_str,
            merged_df=df_all,
            rankings_df=rankings_df,
            Team_Colors=Team_Colors
        )
        # Pass 'standings' to your HTML builder for the Top 25 standings table!
        # (You would do this in assemble_and_send_email, etc.)

    # ...rest of your newsletter build/send logic...
    print(f"Newsletter ready for: {NEWSLETTER_TYPE}")

Newsletter ready for: Big Ten
Newsletter ready for: Big 12
Newsletter ready for: ACC
Newsletter ready for: SEC
DEBUG: Top 25 poll_week being used: 12
Newsletter ready for: Top 25
Newsletter ready for: Ranked GO5


In [33]:
def get_logo_filename(team_name):
    code = team_to_code.get(team_name, None)
    if code is None:
        return ""
    return f"{LOGO_BASE}/{code}.png"


In [34]:
def find_biggest_upset(df):
    df_filtered = df[df['Completed'] == 'Yes'].copy()

    # Assign rank for unranked teams as 26 (assuming top 25 ranked)
    df_filtered['Home_Rank_Fill'] = df_filtered['Home_Rank'].fillna(26).astype(int)
    df_filtered['Away_Rank_Fill'] = df_filtered['Away_Rank'].fillna(26).astype(int)

    if df_filtered.empty:
        return None

    # Step 1: Try to find a ranked upset
    def get_winner_loser_rank(row):
        if int(row['Home_Pts']) > int(row['Away_Pts']):
            return row['Home_Team'], row['Home_Rank_Fill'], row['Away_Team'], row['Away_Rank_Fill']
        else:
            return row['Away_Team'], row['Away_Rank_Fill'], row['Home_Team'], row['Home_Rank_Fill']

    df_filtered[['Winner', 'Winner_Rank', 'Loser', 'Loser_Rank']] = df_filtered.apply(
        lambda r: pd.Series(get_winner_loser_rank(r)), axis=1)

    df_filtered['Rank_Diff'] = df_filtered['Winner_Rank'] - df_filtered['Loser_Rank']
    upset_games = df_filtered[df_filtered['Rank_Diff'] > 0]

    if not upset_games.empty:
        # Biggest upset among ranked games
        biggest_upset = upset_games.loc[upset_games['Rank_Diff'].idxmax()]
        return biggest_upset

    # Step 2: If no ranked upsets, try by win percentage
    def win_pct(record):
        try:
            wins, losses = record.split(' ')[0].split('-')
            wins = int(wins)
            losses = int(losses)
            return wins / (wins + losses) if (wins + losses) > 0 else 0
        except:
            return 0

    df_filtered['Home_Win_Pct'] = df_filtered['Home_Record'].apply(win_pct)
    df_filtered['Away_Win_Pct'] = df_filtered['Away_Record'].apply(win_pct)

    def get_winner_loser_winpct(row):
        if int(row['Home_Pts']) > int(row['Away_Pts']):
            return row['Home_Team'], row['Home_Win_Pct'], row['Away_Team'], row['Away_Win_Pct']
        else:
            return row['Away_Team'], row['Away_Win_Pct'], row['Home_Team'], row['Home_Win_Pct']

    df_filtered[['Winner', 'Winner_Win_Pct', 'Loser', 'Loser_Win_Pct']] = df_filtered.apply(
        lambda r: pd.Series(get_winner_loser_winpct(r)), axis=1)

    # Only count as an upset if winner had a worse record (win pct) than loser
    df_filtered['WinPct_Diff'] = df_filtered['Winner_Win_Pct'] - df_filtered['Loser_Win_Pct']
    winpct_upsets = df_filtered[df_filtered['WinPct_Diff'] < 0]

    if not winpct_upsets.empty:
        # Biggest win pct upset (largest negative difference)
        biggest_winpct_upset = winpct_upsets.loc[winpct_upsets['WinPct_Diff'].idxmin()]
        return biggest_winpct_upset

    # Step 3: No upsets at all
    return None








def build_email_header(game_df, newsletter_type):
    import pandas as pd

    def sup_rank(rank):
        if pd.notnull(rank) and rank != "" and rank != 0:
            return f"<sup style='font-size:12px; color:#000;'>{int(rank)}</sup>&nbsp;"
        return ""

    def pretty_date(date_str):
        try:
            dt = pd.to_datetime(date_str)
            return f"{dt.strftime('%a, %b. ')}{dt.day}"
        except:
            return date_str

    game_df = game_df.copy()
    date_range = game_df['Start_Date'].unique()
    if len(date_range) == 1:
        date_str = f"Games Played: {pretty_date(date_range[0])}"
    else:
        date_str = f"Games Played: {pretty_date(date_range[0])}–{pretty_date(date_range[-1])}"
    n_games = len(game_df)

    title = f"{newsletter_type} Football Daily Recap"
    title_length = len(title)
    if title_length > 28:
        headline_font_size = "40px"
    elif title_length > 23:
        headline_font_size = "43px"
    elif title_length > 18:
        headline_font_size = "45px"
    else:
        headline_font_size = "48px"

    idx_max = game_df['Excitement'].astype(float).idxmax()
    row_max = game_df.loc[idx_max]
    top_game = f"{row_max['Away_Team']} @ {row_max['Home_Team']}"
    top_score = row_max['Excitement']

    biggest_upset = find_biggest_upset(game_df)
    if biggest_upset is not None:
        if int(biggest_upset['Home_Pts']) > int(biggest_upset['Away_Pts']):
            winner_team = biggest_upset['Home_Team']
            winner_rank = biggest_upset.get('Home_Rank', None)
            winner_pts = biggest_upset['Home_Pts']
            loser_team = biggest_upset['Away_Team']
            loser_rank = biggest_upset.get('Away_Rank', None)
            loser_pts = biggest_upset['Away_Pts']
        else:
            winner_team = biggest_upset['Away_Team']
            winner_rank = biggest_upset.get('Away_Rank', None)
            winner_pts = biggest_upset['Away_Pts']
            loser_team = biggest_upset['Home_Team']
            loser_rank = biggest_upset.get('Home_Rank', None)
            loser_pts = biggest_upset['Home_Pts']

        upset_text = (
            f"{sup_rank(winner_rank)}{winner_team} {winner_pts}, "
            f"{sup_rank(loser_rank)}{loser_team} {loser_pts}"
        )
    else:
        upset_text = "No upsets recorded."

    game_df['Margin'] = abs(game_df['Home_Pts'].astype(int) - game_df['Away_Pts'].astype(int))
    idx_blowout = game_df['Margin'].idxmax()
    row_blowout = game_df.loc[idx_blowout]

    def team_display(team_name, rank, pts):
        return f"{sup_rank(rank)}{team_name} {pts}"

    if int(row_blowout['Margin']) >= 17:
        if int(row_blowout['Home_Pts']) >= int(row_blowout['Away_Pts']):
            blowout_game = (
                f"{team_display(row_blowout['Home_Team'], row_blowout.get('Home_Rank', None), row_blowout['Home_Pts'])}, "
                f"{team_display(row_blowout['Away_Team'], row_blowout.get('Away_Rank', None), row_blowout['Away_Pts'])}"
            )
        else:
            blowout_game = (
                f"{team_display(row_blowout['Away_Team'], row_blowout.get('Away_Rank', None), row_blowout['Away_Pts'])}, "
                f"{team_display(row_blowout['Home_Team'], row_blowout.get('Home_Rank', None), row_blowout['Home_Pts'])}"
            )
    else:
        blowout_game = "No blowouts recorded."

    header = f"""
    <div style="padding:16px 0 20px 0; text-align:center; font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif; color: #1B1B1B;">
      <div style="font-size:{headline_font_size}; font-weight:900; letter-spacing:-2px; margin-bottom:10px; color:#004B87; text-shadow: 1px 1px 2px rgba(0,0,0,0.15);">
        {title}
      </div>
      <div style="font-size:20px; font-weight:600; color:#666666; margin-bottom:16px; font-style: italic;">
        {date_str} &mdash; <span style="color:#004B87;">{n_games} Game(s)</span>
      </div>
      <div style="font-size:17px; margin-bottom:10px;">
        <b style="color:#111;">Most Exciting: </b> <span style="font-weight:450;">{top_game}</span>
        <span style="color:#111; font-weight:450;"> (Excitement Score: <b style="font-weight:700; color:#111;">{top_score}</b>)</span>
      </div>
      <div style="font-size:17px; margin-bottom:10px;">
        <b style="color:#111;">Biggest Upset: </b> <span style="font-weight:450; color:#111;">{upset_text}</span>
      </div>
      <div style="font-size:17px;">
        <b style="color:#111;">Biggest Blowout: </b> <span style="font-weight:450; color:#111;">{blowout_game}</span>
      </div>
    </div>
    """
    return header









def make_game_card_email(row, home_cid, away_cid, boxscore_url=None):
    home_color = Team_Colors.get(row['Home_Team'], '#222')
    away_color = Team_Colors.get(row['Away_Team'], '#222')
    home_score_color = home_color
    away_score_color = away_color

    def display_rank(rank, color):
        if pd.notnull(rank):
            return (
                f'<sup style="font-size:12px; color:{color};">{int(rank)}</sup>&nbsp;'
            )
        else:
            return ''

    card = f"""
    <table width="600" cellpadding="0" cellspacing="0" border="0" 
        style="
            background: linear-gradient(to bottom, #fffdf8, #f0f0e9);
            border:2.5px solid #b6b6b6; 
            border-radius:16px; 
            margin-bottom:10px;  /* Tighter spacing below card */
            font-family:'Segoe UI',Arial,sans-serif;
            box-shadow:0 8px 32px #d8d8d8;">
      <tr>
        <!-- Away team -->
        <td width="32%" align="center" style="padding:22px 10px 18px 10px;">
          <img src="cid:{away_cid}" width="64" height="64"
            style="border-radius:50%; border:2.5px solid #c8c8c8; margin-bottom:12px; box-shadow:0 2px 8px #ddd;">
          <div style="font-size:22px; font-weight:600; margin-top:8px; margin-bottom:2px; line-height:1.1; color:{away_color};">
            {display_rank(row['Away_Rank'], away_color)}{row['Away_Team']}
          </div>
          <div style="font-size:12.5px; color:#888; margin-bottom:4px;">
              {row['Away_Record']} ({row['Away_Conf_Record']})
          </div>
          <div style="font-size:30px; font-weight:bold; color:{away_score_color}; margin-top:6px;">{row['Away_Pts']}</div>
        </td>
        <!-- Score box and details -->
        <td width="36%" align="center" style="padding:18px 0 14px 0;">
          <table cellpadding="0" cellspacing="0" border="0"
            style="
              border:1.5px solid #222;
              border-radius:14px;
              font-size:15px;
              width:260px;
              min-width:260px;
              margin:0 auto;
              background: transparent;
              overflow:hidden;">
            <tr style="background-color:#1434A4; color:#fff;">
              <th style="border:none; padding:5px 7px; border-top-left-radius:14px;"></th>
              <th style="border:none; padding:5px 7px;">1</th>
              <th style="border:none; padding:5px 7px;">2</th>
              <th style="border:none; padding:5px 7px;">3</th>
              <th style="border:none; padding:5px 7px;">4</th>
              <th style="border:none; padding:5px 16px; border-top-right-radius:14px;">F</th>
            </tr>
            <tr style="background-color:#fff;">
              <td align="center" style="border:none; padding:4px 7px; font-size:14px; color:{away_color};">{row['Away_Team']}</td>
              <td align="center" style="border:none;">{row['Away_1Q']}</td>
              <td align="center" style="border:none;">{row['Away_2Q']}</td>
              <td align="center" style="border:none;">{row['Away_3Q']}</td>
              <td align="center" style="border:none;">{row['Away_4Q']}</td>
              <td align="center" style="border:none; font-weight:bold; padding:4px 16px;">{row['Away_Pts']}</td>
            </tr>
            <tr style="background-color:#f5fbff;">
              <td align="center" style="border:none; padding:4px 7px; font-size:14px; color:{home_color}; border-bottom-left-radius:14px;">{row['Home_Team']}</td>
              <td align="center" style="border:none;">{row['Home_1Q']}</td>
              <td align="center" style="border:none;">{row['Home_2Q']}</td>
              <td align="center" style="border:none;">{row['Home_3Q']}</td>
              <td align="center" style="border:none;">{row['Home_4Q']}</td>
              <td align="center" style="border:none; font-weight:bold; padding:4px 16px; border-bottom-right-radius:14px;">{row['Home_Pts']}</td>
            </tr>
          </table>
          <div style="margin-top:10px; color:#444; font-size:13.5px; line-height:1.5;">
            <div style="margin:0; padding:0; font-size:13.5px;">{row['Start_Date']} &bull; {row['Start_Time']} ET</div>
            <div style="
              font-weight:bold;
              margin:0;
              padding:0;
              font-size:14px;
              text-align:center;
              display:inline-block;
              max-width:220px;
              white-space:nowrap;
              overflow:hidden;
              text-overflow:ellipsis;
            ">{row['Venue']}</div>
            <div style="margin:0; padding:0; font-size:13.5px;">
              <span style="font-weight:bold;">{row['Conference_Game']}</span> | Excitement: <b>{row['Excitement']}</b>
            </div>
          </div>
        </td>
        <!-- Home team -->
        <td width="32%" align="center" style="padding:22px 10px 18px 10px;">
          <img src="cid:{home_cid}" width="64" height="64"
            style="border-radius:50%; border:2.5px solid #c8c8c8; margin-bottom:12px; box-shadow:0 2px 8px #ddd;">
          <div style="font-size:22px; font-weight:600; margin-top:8px; margin-bottom:2px; line-height:1.1; color:{home_color};">
            {display_rank(row['Home_Rank'], home_color)}{row['Home_Team']}
          </div>
          <div style="font-size:12.5px; color:#888; margin-bottom:4px;">
              {row['Home_Record']} ({row['Home_Conf_Record']})
          </div>
          <div style="font-size:30px; font-weight:bold; color:{home_score_color}; margin-top:6px;">{row['Home_Pts']}</div>
        </td>
      </tr>
    </table>
    """

    # ---- Add Centered, Small Box Score Link If Available ----
    if boxscore_url:
        card += f"""
        <div style="text-align:center; margin:4px 0 8px 0; font-size:13px;">
          <a href="{boxscore_url}" target="_blank"
             style="color:#1478b6;font-weight:500;font-size:13px;letter-spacing:0.2px;text-decoration:underline;">
            View Full Box Score &amp; Stats
          </a>
        </div>
        """

    return card








def build_email_footer(df_games, all_games, newsletter_type, rankings_df=None, poll_week=None):
    import numpy as np
    import pandas as pd

    # For conferences other than SEC, get unique Conference_Game values from df_games
    if newsletter_type == 'SEC':
        # Strictly filter SEC games for SEC newsletter
        filtered_games = all_games[
            (all_games['Home_Conference'] == 'SEC') | (all_games['Away_Conference'] == 'SEC')
        ]
    else:
        confs = df_games['Conference_Game'].dropna().unique()
        filtered_games = all_games[all_games['Conference_Game'].isin(confs)]

    # Find next date after current batch
    current_max = max(pd.to_datetime(df_games['Start_Date']))
    all_dates = sorted(set(pd.to_datetime(filtered_games['Start_Date'])))
    next_dates = [d for d in all_dates if d > current_max]

    # Helper to calculate win percentage from record string
    def win_pct(record):
        try:
            wins, losses = record.split(' ')[0].split('-')
            wins = int(wins)
            losses = int(losses)
            return wins / (wins + losses) if (wins + losses) > 0 else 0
        except:
            return 0

    # Helper to format team with rank superscript
    def team_with_rank(team_name, rank):
        if pd.notnull(rank) and rank != "":
            return (
                f'<sup style="font-size:14px; vertical-align: super; margin-right: 3px; color:#003366;">{int(rank)}</sup>'
                f'<span style="font-size:18px;">{team_name}</span>'
            )
        else:
            return f'<span style="font-size:18px;">{team_name}</span>'

    if next_dates:
        next_day = next_dates[0].strftime("%a, %b %d, %Y")
        # Add period to month abbreviation for style consistency
        months_with_period = {"Jan", "Feb", "Mar", "Apr", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"}
        for m in months_with_period:
            if f" {m} " in next_day:
                next_day = next_day.replace(f" {m} ", f" {m}. ")
                break

        # Filter games on the next date
        next_games = filtered_games[
            pd.to_datetime(filtered_games['Start_Date']) == next_dates[0]
        ]

        # If SEC newsletter, ensure next_games only include SEC teams
        if newsletter_type == 'SEC':
            next_games = next_games[
                (next_games['Home_Conference'] == 'SEC') | (next_games['Away_Conference'] == 'SEC')
            ]

        # Identify best upcoming game for Top 25: the ranked matchup with smallest average rank,
        # else best individual ranked team, else fallback to combined win pct
        ranked_games = next_games[
            next_games['Home_Rank'].notnull() & next_games['Away_Rank'].notnull()
        ].copy()

        candidates = pd.DataFrame()

        if not ranked_games.empty:
            ranked_games['Avg_Rank'] = (ranked_games['Home_Rank'] + ranked_games['Away_Rank']) / 2
            candidates = ranked_games.sort_values('Avg_Rank')
        else:
            one_rank_games = next_games[
                (next_games['Home_Rank'].notnull() & next_games['Away_Rank'].isnull()) |
                (next_games['Home_Rank'].isnull() & next_games['Away_Rank'].notnull())
            ].copy()
            if not one_rank_games.empty:
                def best_ind_rank(row):
                    ranks = [r for r in [row['Home_Rank'], row['Away_Rank']] if pd.notnull(r)]
                    return min(ranks) if ranks else np.nan
                one_rank_games['Best_Rank'] = one_rank_games.apply(best_ind_rank, axis=1)
                candidates = one_rank_games.sort_values('Best_Rank')
            else:
                # No ranked teams
                if newsletter_type == 'Top 25':
                    candidates = pd.DataFrame()  # No best game shown
                else:
                    next_games = next_games.copy()
                    next_games['Home_Win_Pct'] = next_games['Home_Record'].apply(win_pct)
                    next_games['Away_Win_Pct'] = next_games['Away_Record'].apply(win_pct)
                    next_games['Combined_Win_Pct'] = next_games['Home_Win_Pct'] + next_games['Away_Win_Pct']
                    candidates = next_games.sort_values('Combined_Win_Pct', ascending=False)

        # Select best candidate game, or empty if none
        if not candidates.empty:
            sort_column = (
                'Avg_Rank' if 'Avg_Rank' in candidates.columns else
                'Best_Rank' if 'Best_Rank' in candidates.columns else
                'Combined_Win_Pct'
            )
            top_val = candidates.iloc[0][sort_column]
            top_candidates = candidates[candidates[sort_column] == top_val]
            best_game = top_candidates.sample(1).iloc[0]
            best_game_text = (
                f"{team_with_rank(best_game['Away_Team'], best_game['Away_Rank'])} "
                f"vs {team_with_rank(best_game['Home_Team'], best_game['Home_Rank'])}"
            )
            best_game_line = f"<b>Best Upcoming Game: </b> {best_game_text}"
        else:
            best_game_line = ""

        n_next = len(next_games)
        next_line = f"<b>Next Game(s):</b> {next_day} — {n_next} Scheduled"
    else:
        next_line = "<b>No more games scheduled.</b>"
        best_game_line = ""

    footer = f"""
    <div style="margin-top:12px; padding:12px 0 32px 0; text-align:center; color:#000000; font-size:18px;">
      {next_line}<br>
      {best_game_line}<br><br>
      <span style="color:#000080; font-size:18px; font-family: 'Arial Black', Gadget, sans-serif;">Thank You for Reading!</span>
      <br>
      <span style="color:#888; font-size:14px; display:block; margin-top:7px;">{NEWSLETTER_AUTHOR}</span>
    </div>
    """
    return footer











def build_standings_table(
    newsletter_type, games_yesterday, all_games_df, send_date_str, Team_Colors,
    standings_override=None  # Optional override for Top 25 standings
):
    import numpy as np
    import pandas as pd

    def record_tuple(rec_str):
        try:
            main_part = rec_str.split(" ")[0]
            wins, losses = map(int, main_part.split("-"))
            return wins, losses
        except:
            return 0, 0

    def format_date_short(date_str):
        try:
            dt = pd.to_datetime(date_str)
            return dt.strftime('%a, %b. ') + str(dt.day)
        except:
            return ""

    def get_team_rank(team, date, all_games_df):
        games = all_games_df[((all_games_df['Home_Team'] == team) | (all_games_df['Away_Team'] == team))]
        games = games[pd.to_datetime(games['Start_Date']) <= pd.to_datetime(date)]
        if games.empty:
            return ""
        last = games.sort_values('Start_Date', ascending=False).iloc[0]
        if last['Home_Team'] == team and pd.notnull(last['Home_Rank']):
            return int(last['Home_Rank'])
        elif last['Away_Team'] == team and pd.notnull(last['Away_Rank']):
            return int(last['Away_Rank'])
        return ""

    if newsletter_type == "Ranked GO5":
        return ""

    # === Get standings list ===
    if newsletter_type == "Top 25" and standings_override is not None:
        standings = standings_override
        table_title = "Top 25 Standings"
        conf_wl_label = ""  # No conf record for Top 25
    elif newsletter_type == "Top 25":
        # fallback if no override (less ideal)
        card_teams = set()
        for _, row in games_yesterday.iterrows():
            if pd.notnull(row['Home_Rank']) and row['Home_Rank'] <= 25:
                card_teams.add((row['Home_Team'], int(row['Home_Rank'])))
            if pd.notnull(row['Away_Rank']) and row['Away_Rank'] <= 25:
                card_teams.add((row['Away_Team'], int(row['Away_Rank'])))
        ranked_teams = sorted(card_teams, key=lambda x: x[1])
        standings = []
        for team, rank in ranked_teams:
            row_home = games_yesterday[games_yesterday['Home_Team'] == team]
            row_away = games_yesterday[games_yesterday['Away_Team'] == team]
            if not row_home.empty:
                overall_record = row_home.iloc[0]['Home_Record']
                conf_record = ""  # No conf record for Top 25
            elif not row_away.empty:
                overall_record = row_away.iloc[0]['Away_Record']
                conf_record = ""
            else:
                overall_record = ""
                conf_record = ""
            future_games = all_games_df[
                ((all_games_df['Home_Team'] == team) | (all_games_df['Away_Team'] == team)) &
                (pd.to_datetime(all_games_df['Start_Date']) > pd.to_datetime(send_date_str))
            ].sort_values('Start_Date')
            if not future_games.empty:
                next_game = future_games.iloc[0]
                if next_game['Home_Team'] == team:
                    opp = next_game['Away_Team']
                else:
                    opp = next_game['Home_Team']
                opp_rank = get_team_rank(opp, next_game['Start_Date'], all_games_df)
                venue = "" if next_game['Home_Team'] == team else "@"
                next_game_str = f"{venue}{opp}"
                next_game_rank = opp_rank
                next_game_date = next_game['Start_Date']
            else:
                next_game_str = ""
                next_game_rank = ""
                next_game_date = ""
            standings.append({
                "Team": team,
                "Rank": rank,
                "Overall_Record": overall_record,
                "Conf_Record": conf_record,
                "Next_Game": next_game_str,
                "Next_Game_Rank": next_game_rank,
                "Next_Game_Date": next_game_date
            })
        table_title = "Top 25 Standings"
        conf_wl_label = ""
    else:
        # Conferences and other newsletters
        if newsletter_type in ["Big Ten", "Big 12", "SEC"]:
            teams = sorted(
                pd.unique(
                    pd.concat([
                        all_games_df.loc[all_games_df['Home_Conference'] == newsletter_type, 'Home_Team'],
                        all_games_df.loc[all_games_df['Away_Conference'] == newsletter_type, 'Away_Team'],
                    ])
                )
            )
        elif newsletter_type == "ACC":
            teams = sorted(
                t for t in pd.unique(
                    pd.concat([
                        all_games_df.loc[all_games_df['Home_Conference'] == "ACC", 'Home_Team'],
                        all_games_df.loc[all_games_df['Away_Conference'] == "ACC", 'Away_Team'],
                    ])
                ) if t != "Notre Dame"
            )
        else:
            return ""

        standings = []
        for team in teams:
            mask_home = (all_games_df['Home_Team'] == team)
            mask_away = (all_games_df['Away_Team'] == team)
            games_team = all_games_df[(mask_home | mask_away)]
            games_team = games_team[pd.to_datetime(games_team['Start_Date']) <= pd.to_datetime(send_date_str)]
            if not games_team.empty:
                last_game = games_team.sort_values('Start_Date', ascending=False).iloc[0]
                if last_game['Home_Team'] == team:
                    overall_record = last_game['Home_Record']
                    conf_record = last_game['Home_Conf_Record']
                    rank = int(last_game['Home_Rank']) if pd.notnull(last_game['Home_Rank']) else ""
                else:
                    overall_record = last_game['Away_Record']
                    conf_record = last_game['Away_Conf_Record']
                    rank = int(last_game['Away_Rank']) if pd.notnull(last_game['Away_Rank']) else ""
            else:
                overall_record, conf_record, rank = "", "", ""
            future_games = all_games_df[
                ((all_games_df['Home_Team'] == team) | (all_games_df['Away_Team'] == team)) &
                (pd.to_datetime(all_games_df['Start_Date']) > pd.to_datetime(send_date_str))
            ].sort_values('Start_Date')
            if not future_games.empty:
                next_game = future_games.iloc[0]
                if next_game['Home_Team'] == team:
                    opp = next_game['Away_Team']
                else:
                    opp = next_game['Home_Team']
                opp_rank = get_team_rank(opp, next_game['Start_Date'], all_games_df)
                venue = "" if next_game['Home_Team'] == team else "@"
                next_game_str = f"{venue}{opp}"
                next_game_rank = opp_rank
                next_game_date = next_game['Start_Date']
            else:
                next_game_str = ""
                next_game_rank = ""
                next_game_date = ""
            standings.append({
                "Team": team,
                "Rank": rank,
                "Overall_Record": overall_record,
                "Conf_Record": conf_record,
                "Next_Game": next_game_str,
                "Next_Game_Rank": next_game_rank,
                "Next_Game_Date": next_game_date
            })

        def safe_split(rec):
            try:
                w, l = map(int, rec.split()[0].split('-'))
                return w, l
            except:
                return 0, 99
        standings = sorted(
            standings,
            key=lambda r: (
                -safe_split(r['Conf_Record'])[0],   # Conf Wins DESC
                safe_split(r['Conf_Record'])[1],    # Conf Losses ASC
                -safe_split(r['Overall_Record'])[0],# Overall Wins DESC
                safe_split(r['Overall_Record'])[1], # Overall Losses ASC
                r['Rank'] if r['Rank'] != "" else 99  # Rank ASC
            )
        )
        table_title = f"{newsletter_type} Standings"
        conf_wl_label = f"{newsletter_type} W-L" if newsletter_type != "ACC" else "ACC W-L"

    # Set columns based on newsletter type
    if newsletter_type == "Top 25":
        col_labels = ["Team", "W-L", "Next Opp.", "Date"]  # No conf record or rank column
    else:
        col_labels = ["Team", "W-L", conf_wl_label, "Next Opp.", "Date"]

    table_html = f"""
    <table width="100%" cellpadding="0" cellspacing="0" border="0" align="center"
      style="background:#f9f9f9;
             font-family:'Segoe UI',Arial,sans-serif;
             font-size:15px;
             line-height:1.45;
             margin:0 auto 0 auto;">
      <tr>
        <td colspan="{len(col_labels)}" align="center" style="font-size:22px;font-weight:bold;padding:10px 8px 4px 8px;color:#003366;">
          {table_title}
        </td>
      </tr>
      <tr style="background:#e7ebf7;">
        {''.join([f'<th style="padding:8px 8px;font-size:15px;color:#222;font-weight:700;text-align:center;">{col}</th>' for col in col_labels])}
      </tr>
    """

    for idx, s in enumerate(standings):
        team_display = clean_team_name(s["Team"])
        rank = s.get("Rank", "")
        team_color = Team_Colors.get(team_display, "#163868")
        ow, ol = record_tuple(s["Overall_Record"])
        wl_str = f"{ow}-{ol}" if (ow + ol) > 0 else "0-0"
    
        # Compose team name with rank as superscript, rank superscript in team color
        if rank not in ["", 0, None, np.nan]:
            if isinstance(rank, (int, float)) or (isinstance(rank, str) and str(rank).isdigit()):
                rank_sup = f'<sup style="font-size:11px;vertical-align:super;color:{team_color};margin-right:4px;">{rank}</sup>'
            else:
                rank_sup = ""
        else:
            rank_sup = ""
        team_name_html = f'{rank_sup}{team_display}'
    
        row_cells = [
            f'<td style="padding:7px 8px;font-size:15px;font-weight:600;color:{team_color};text-align:left;">{team_name_html}</td>',
            f'<td style="padding:7px 4px;font-size:15px;text-align:center;">{wl_str}</td>'
        ]
    
        if newsletter_type != "Top 25":
            cw, cl = record_tuple(s.get("Conf_Record", ""))
            cwl_str = f"{cw}-{cl}" if (cw + cl) > 0 else "0-0"
            row_cells.append(f'<td style="padding:7px 4px;font-size:15px;text-align:center;">{cwl_str}</td>')
    
        opp = s.get("Next_Game", "")
        opp_rank = s.get("Next_Game_Rank", "")
        if opp and opp_rank not in ["", 0, "0", None, np.nan]:
            if isinstance(opp_rank, (int, float)) or (isinstance(opp_rank, str) and str(opp_rank).isdigit()):
                opp_sup = f'<sup style="font-size:11px;vertical-align:super;color:#444;margin-right:2px;">{opp_rank}</sup>'
            else:
                opp_sup = ""
        else:
            opp_sup = ""
        next_opp_html = f'{opp_sup}{opp.replace("@", "<span style=\'font-weight:600\'>&#64;</span>")}' if opp else ""
    
        date_fmt = format_date_short(s.get("Next_Game_Date", "")) if opp else ""
        row_bg = "#f4f6fa" if idx % 2 == 0 else "#ffffff"
    
        row_cells += [
            f'<td style="padding:7px 8px;font-size:15px;text-align:center;">{next_opp_html}</td>',
            f'<td style="padding:7px 8px;font-size:15px;text-align:center;">{date_fmt}</td>'
        ]
    
        table_html += f"<tr style='background:{row_bg};'>{''.join(row_cells)}</tr>"

    table_html += "</table>"
    return table_html


In [40]:
def ordinal(n):
    n = int(n)
    if 10 <= n % 100 <= 20:
        suffix = 'th'
    else:
        suffix = {1: 'st', 2: 'nd', 3: 'rd'}.get(n % 10, 'th')
    return str(n) + f'<sup style="font-size:11px;">{suffix}</sup>'

def format_date(date_str):
    dt = pd.to_datetime(date_str)
    return f"{dt.strftime('%a')}, {dt.strftime('%b')} {ordinal(dt.day)}, {dt.year}"

In [60]:
def make_full_game_html(
    game_row,
    Team_Colors=None,
    home_logo_url=None,
    away_logo_url=None,
    home_rank=None,
    away_rank=None,
):
    import pandas as pd
    from datetime import datetime

    font_links = """
    <link href="https://fonts.googleapis.com/css?family=Oswald:700&display=swap" rel="stylesheet">
    <link href="https://fonts.googleapis.com/css?family=Montserrat:700&display=swap" rel="stylesheet">
    """

    def format_friendly_date(date_str):
        dt = pd.to_datetime(date_str)
        return dt.strftime('%a, %b. ') + str(dt.day) + dt.strftime(', %Y')

    def format_time(tstr):
        try:
            t = datetime.strptime(str(tstr), "%H:%M")
            return t.strftime("%I:%M %p").lstrip("0")
        except:
            return tstr

    def team_rank_sup(rank):
        if pd.notnull(rank) and str(rank).isdigit() and int(rank) <= 25:
            return f'<sup style="font-size:1.1em; color:#3366cc; vertical-align:super; font-family:Montserrat,Arial,sans-serif;">{int(rank)}</sup>&nbsp;'
        else:
            return ""

    def stat_row(label, away_key, home_key):
        away_val = game_row.get(f'Away_{away_key}', '')
        home_val = game_row.get(f'Home_{home_key}', '')
        return f"""
        <tr>
            <td class="away">{away_val if pd.notnull(away_val) else '-'}</td>
            <td class="stat">{label}</td>
            <td class="home">{home_val if pd.notnull(home_val) else '-'}</td>
        </tr>
        """

    def section_row(label):
        return f"""
        <tr class="section-header"><td colspan="3">{label}</td></tr>
        """

    offense_stats = [
        ("Completion Attempts", "Completion_Attempts"),
        ("Passing TDs", "Passing_TDs"),
        ("Net Passing Yards", "Net_Passing_Yards"),
        ("Rushing Attempts", "Rushing_Attempts"),
        ("Rushing Yards", "Rushing_Yards"),
        ("Rushing TDs", "Rushing_TDs"),
        ("Yards Per Rush", "Yards_Per_Rush_Attempt"),
        ("First Downs", "First_Downs"),
        ("Third Down Eff.", "Third_Down_Eff"),
        ("Fourth Down Eff.", "Fourth_Down_Eff"),
        ("Total Yards", "Total_Yards"),
        ("Points", "Points"),
        ("Possession Time", "Possession_Time"),
        ("Total Fumbles", "Total_Fumbles"),
        ("Fumbles Lost", "Fumbles_Lost"),
        ("Turnovers", "Turnovers"),
        ("Total Penalties Yards", "Total_Penalties_Yards"),
    ]
    defense_stats = [
        ("Sacks", "Sacks"),
        ("Tackles", "Tackles"),
        ("Tackles For Loss", "Tackles_For_Loss"),
        ("Interceptions", "Interceptions"),
        ("Passes Deflected", "Passes_Deflected"),
        ("Interception TDs", "Interception_TDs"),
        ("Interception Yards", "Interception_Yards"),
        ("QB Hurries", "Qb_Hurries"),
        ("Defensive TDs", "Defensive_TDs"),
        ("Fumbles Recovered", "Fumbles_Recovered"),
        ("Passes Intercepted", "Passes_Intercepted"),
    ]
    special_teams_stats = [
        ("Kicking Points", "Kicking_Points"),
        ("Kick Return Yards", "Kick_Return_Yards"),
        ("Kick Returns", "Kick_Returns"),
        ("Kick Return TDs", "Kick_Return_TDs"),
        ("Punt Return Yards", "Punt_Return_Yards"),
        ("Punt Returns", "Punt_Returns"),
        ("Punt Return TDs", "Punt_Return_TDs"),
    ]

    home = game_row['Home_Team']
    away = game_row['Away_Team']
    home_rank_val = home_rank if home_rank is not None else game_row.get('Home_Rank', '')
    away_rank_val = away_rank if away_rank is not None else game_row.get('Away_Rank', '')
    home_color = Team_Colors.get(home, '#1452a2') if Team_Colors else '#1452a2'
    away_color = Team_Colors.get(away, '#222') if Team_Colors else '#222'
    main_blue = "#1452a2"

    # Format date and time for display
    display_date = format_friendly_date(game_row['Start_Date'])
    display_time = format_time(game_row['Start_Time'])

    # ---- HEADER CARD ----
    header_html = f"""
    <div style="display:flex;justify-content:center;">
      <div style="background:#fff;border:3px solid {main_blue};border-radius:18px;box-shadow:0 8px 40px #b9d3ea;
          margin:24px 0 24px 0;max-width:760px;width:98vw;padding:24px 32px 16px 32px;position:relative;">
        <div style="display:flex;align-items:center;justify-content:space-between;">
          <!-- Away team -->
          <div style="flex:1;text-align:center;">
            <img src="{away_logo_url}" style="height:74px;width:74px;border-radius:14px;border:2.5px solid #d8d8d8;">
            <div style="font-family:'Oswald',Arial,sans-serif;font-size:2em;font-weight:900;letter-spacing:0.5px; margin-top:12px;color:{away_color};">
              {team_rank_sup(away_rank_val)}{away}
            </div>
            <div style="color:#888;font-size:1em;font-weight:500;margin-top:2px;">
              {game_row['Away_Record']} ({game_row['Away_Conf_Record']})
            </div>
          </div>
          <!-- Scorebox Center -->
          <div style="flex:1.1;display:flex;flex-direction:column;align-items:center;justify-content:center;min-width:155px;max-width:210px;">
            <div style="background:{main_blue};color:#fff;border-radius:14px;box-shadow:0 1.5px 8px #bbdbf6;
                padding:14px 18px 13px 18px; width:auto;margin-bottom:5px;display:flex;flex-direction:column;align-items:center;">
              <div style="display:flex;align-items:center;justify-content:center;width:100%;margin-bottom:6px;">
                <span style="font-family:'Montserrat',Arial,sans-serif;font-size:2.1em;font-weight:900;padding:0 12px;">
                    {game_row['Away_Pts']}
                </span>
                <span style="font-size:1.45em;font-family:'Montserrat',Arial,sans-serif;font-weight:900;">–</span>
                <span style="font-family:'Montserrat',Arial,sans-serif;font-size:2.1em;font-weight:900;padding:0 12px;">
                    {game_row['Home_Pts']}
                </span>
              </div>
              <div style="font-size:0.95em;font-family:'Montserrat',Arial,sans-serif;font-weight:500;color:#f7faff;text-align:center;">
                {display_date}
              </div>
              <div style="font-size:0.92em;font-family:'Montserrat',Arial,sans-serif;font-weight:400;color:#e6edfa;text-align:center;">
                {display_time} ET
              </div>
            </div>
            <div style="font-size:1.06em;color:#222;font-weight:700;letter-spacing:0.15px;text-align:center;">
              {game_row['Venue']}
            </div>
            <div style="font-size:1em;color:#444;font-weight:500;text-align:center;">
              <span style="font-weight:700;">{game_row['Conference_Game']}</span>
              &nbsp;|&nbsp; Excitement: <b>{game_row.get('Excitement','')}</b>
            </div>
          </div>
          <!-- Home team -->
          <div style="flex:1;text-align:center;">
            <img src="{home_logo_url}" style="height:74px;width:74px;border-radius:14px;border:2.5px solid #d8d8d8;">
            <div style="font-family:'Oswald',Arial,sans-serif;font-size:2em;font-weight:900;letter-spacing:0.5px; margin-top:12px;color:{home_color};">
              {team_rank_sup(home_rank_val)}{home}
            </div>
            <div style="color:#888;font-size:1em;font-weight:500;margin-top:2px;">
              {game_row['Home_Record']} ({game_row['Home_Conf_Record']})
            </div>
          </div>
        </div>
      </div>
    </div>
    """

    # ---- STATS TABLE ----
    table_html = f"""
    <div class="table-outer">
    <table class="stat-table">
        <tr class="table-header">
            <th class="away" style="color:{away_color}; text-align:center; font-family:'Oswald',Arial,sans-serif; background:#fff;">
                <span style="font-size:1.2em; font-weight:900;">{team_rank_sup(away_rank_val)}{away}</span>
            </th>
            <th class="stat"></th>
            <th class="home" style="color:{home_color}; text-align:center; font-family:'Oswald',Arial,sans-serif; background:#fff;">
                <span style="font-size:1.2em; font-weight:900;">{team_rank_sup(home_rank_val)}{home}</span>
            </th>
        </tr>
        {section_row("Offense")}
        {''.join([stat_row(label, key, key) for label, key in offense_stats])}
        {section_row("Defense")}
        {''.join([stat_row(label, key, key) for label, key in defense_stats])}
        {section_row("Special Teams")}
        {''.join([stat_row(label, key, key) for label, key in special_teams_stats])}
    </table>
    </div>
    """

    # ---- CSS ----
    css = f"""
    <style>
    body {{
        background: #eaf3fa;
    }}
    .cfb-table-wrap .game-card {{
        background: #f6fbfc;
        border: 2.5px solid #1452a2;
        border-radius: 16px;
        padding: 16px 10px 12px 10px;
        width: 95%;
        max-width: 800px;
        margin: 24px auto 18px auto;
        box-shadow: 0 6px 18px #d8eaf6;
        font-family: 'Segoe UI', Arial, sans-serif;
        text-align:center;
    }}
    .cfb-table-wrap .stat-table {{
        border-collapse: separate;
        border-spacing: 0;
        margin: 0 auto 40px auto;
        width: 570px;
        max-width: 98vw;
        font-family: 'Segoe UI', Arial, sans-serif;
        background: #fff;
        border: 2.5px solid #1452a2;
        border-radius: 16px;
        box-shadow: 0 6px 18px #d8eaf6;
        overflow: hidden;
        text-align: center;
    }}
    .cfb-table-wrap .stat-table th, .cfb-table-wrap .stat-table td {{
        font-size: 1.11em;
        text-align: center;
        border: none;
    }}
    .cfb-table-wrap .stat-table th.stat, .cfb-table-wrap .stat-table td.stat {{
        min-width: 94px;
        max-width: 105px;
        width: 98px;
        background: #f2f2f3;
        color: #111;
        text-align: center;
        font-weight: 500;
    }}
    .cfb-table-wrap .stat-table td.away, .cfb-table-wrap .stat-table th.away, .cfb-table-wrap .stat-table td.home, .cfb-table-wrap .stat-table th.home {{
        width: 190px;
        min-width: 110px;
        max-width: 220px;
        background: #fff;
        font-weight: 700;
        text-align: center;
        font-size: 1.25em;
        vertical-align: middle;
    }}
    .cfb-table-wrap .stat-table .section-header {{
        background: #e6f1fc !important;
        font-weight: 700;
        font-size: 1.09em;
        color: #175886;
        text-align: center;
        border-top: none;
        border-bottom: none;
        letter-spacing: 0.6px;
    }}
    .cfb-table-wrap .stat-table tr.table-header th {{
        background: #e3edfa;
        color: #111;
        font-weight: 700;
        font-size: 1.19em;
        border: none;
        text-align: center;
        vertical-align: middle;
    }}
    .cfb-table-wrap .stat-table tr {{
        border: none;
    }}
    .cfb-table-wrap .stat-table td, .cfb-table-wrap .stat-table th {{
        border: none;
    }}
    </style>
    """

    html = f'{font_links}<div class="cfb-table-wrap">{header_html}{table_html}</div>{css}'
    return html


In [61]:
from IPython.display import display, HTML
game_row = df_all.iloc[0]
home_logo_url = get_logo_filename(game_row['Home_Team'])
away_logo_url = get_logo_filename(game_row['Away_Team'])
html_code = make_full_game_html(
    game_row,
    Team_Colors=Team_Colors,
    home_logo_url=home_logo_url,
    away_logo_url=away_logo_url,
)
display(HTML(html_code))


In [36]:
def send_email_with_logos(subject, html_body, logo_files, recipients, sender,
                          smtp_server, smtp_port, smtp_user, smtp_password):
    try:
        # Create email message container
        msg = MIMEMultipart('related')
        msg['Subject'] = subject
        msg['From'] = sender
        msg['To'] = ', '.join(recipients)

        # Create the HTML part (multipart/alternative for compatibility)
        msg_alternative = MIMEMultipart('alternative')
        msg.attach(msg_alternative)

        # Attach the HTML content
        msg_text = MIMEText(html_body, 'html')
        msg_alternative.attach(msg_text)

        # Attach each logo as inline image with Content-ID
        for cid, file_path in logo_files:
            if os.path.isfile(file_path):
                with open(file_path, 'rb') as img:
                    img_data = img.read()
                    img_mime = MIMEImage(img_data)
                    img_mime.add_header('Content-ID', f'<{cid}>')
                    img_mime.add_header('Content-Disposition', 'inline', filename=os.path.basename(file_path))
                    msg.attach(img_mime)
            else:
                print(f"Logo file not found: {file_path}")

        # Send the email using SMTP SSL
        with smtplib.SMTP_SSL(smtp_server, smtp_port) as server:
            server.login(smtp_user, smtp_password)
            server.sendmail(sender, recipients, msg.as_string())

        print("Email sent successfully.")
        return True

    except Exception as e:
        print("Failed to send email.")
        print("Error:", e)
        return False


In [37]:
def assemble_and_send_email(newsletter_type, all_games_df, send_date_str, rankings_df=None):
    # Filter games for this newsletter type
    games = filter_games(all_games_df, newsletter_type)
    # Only completed games from the date
    games_yesterday = games[(games['Completed'] == 'Yes') & (games['Start_Date'] == send_date_str)].copy()
    if games_yesterday.empty:
        print(f"[{newsletter_type}] No {newsletter_type} completed games on {send_date_str}. No email will be sent.")
        return False
    
    # Ranking/sorting logic
    def min_rank(row):
        ranks = []
        if pd.notnull(row['Home_Rank']): ranks.append(row['Home_Rank'])
        if pd.notnull(row['Away_Rank']): ranks.append(row['Away_Rank'])
        return min(ranks) if ranks else np.nan

    games_yesterday['Min_Rank'] = games_yesterday.apply(min_rank, axis=1)
    games_yesterday['Min_Rank_Fill'] = games_yesterday['Min_Rank'].fillna(9999)
    games_yesterday_sorted = games_yesterday.sort_values(
        by=['Min_Rank_Fill', 'Excitement'], ascending=[True, False]
    ).reset_index(drop=True)

    logo_files = []
    cid_lookup = {}
    report_html = build_email_header(games_yesterday_sorted, newsletter_type)
    game_card_blocks = []

    # ---- BATCH MODE: Write all files, collect URLs ----
    for idx, (_, row) in enumerate(games_yesterday_sorted.iterrows(), 1):
        home_team = row['Home_Team']
        away_team = row['Away_Team']
        home_logo = get_logo_filename(home_team)
        away_logo = get_logo_filename(away_team)
        for team, logo_file, label in [(away_team, away_logo, 'away'), (home_team, home_logo, 'home')]:
            if logo_file is None or not os.path.isfile(logo_file): continue
            if (team, label) not in cid_lookup:
                cid = f"logo_{team.replace(' ', '').replace('&','').replace('(','').replace(')','')[:12]}_{label}"
                cid_lookup[(team, label)] = cid
                logo_files.append((cid, logo_file))
        home_cid = cid_lookup.get((home_team, 'home'), '')
        away_cid = cid_lookup.get((away_team, 'away'), '')
    
        # --- Write full HTML file to the subfolder ---
        game_id = row['Game_ID']
        boxscore_filename = f"{game_id}_boxscore.html"
        # Point to the subfolder inside the repo
        html_subfolder = os.path.join(REPO_DIR, "cfb_boxscores")
        os.makedirs(html_subfolder, exist_ok=True)  # Make sure subfolder exists
    
        file_path = os.path.join(html_subfolder, boxscore_filename)
        full_game_html = make_full_game_html(
            row,
            Team_Colors=Team_Colors,
            home_logo_url=home_logo,
            away_logo_url=away_logo,
        )
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(full_game_html)
    
        # Generate public GitHub Pages URL (note the /cfb_boxscores/ in URL!)
        boxscore_url = f"https://semmertime.github.io/cfb_boxscores/cfb_boxscores/{boxscore_filename}"
    
        # --- Add card block for newsletter (with link if desired) ---
        card_html = make_game_card_email(row, home_cid, away_cid, boxscore_url=boxscore_url)
        game_card_blocks.append(
            '<table width="100%" border="0" cellspacing="0" cellpadding="0" style="margin:0 auto;"><tr><td align="center">'
            + card_html +
            '</td></tr></table>'
        )
    
    # --- BATCH GIT ADD/COMMIT/PUSH (only once, after loop) ---
    try:
        subprocess.run([GIT_PATH, "add", "."], cwd=REPO_DIR, check=True)
        subprocess.run([GIT_PATH, "commit", "-m", "Batch update boxscore HTML files"], cwd=REPO_DIR, check=True)
        subprocess.run([GIT_PATH, "push"], cwd=REPO_DIR, check=True)
        print("✅ All boxscore files committed and pushed in one batch.")
    except Exception as e:
        print("❌ Git batch push failed:", e)
    
    # --- Combine all card HTMLs into report_html ---
    report_html += "\n".join(game_card_blocks)

    # --- STANDINGS HTML ---
    if newsletter_type == "Top 25":
        if rankings_df is None:
            raise ValueError("rankings_df must be provided for Top 25 newsletter")
        # Determine poll week using your logic (safe, automatic)
        games_dates = pd.to_datetime(all_games_df['Start_Date'])
        yesterday = pd.to_datetime(send_date_str)
        games_before = all_games_df[games_dates <= yesterday]
        week_for_poll = pd.to_numeric(games_before['Week'], errors='coerce').max()
        available_poll_weeks = sorted(rankings_df['Week'].unique())
        if pd.isnull(week_for_poll) or week_for_poll < min(available_poll_weeks):
            poll_week = min(available_poll_weeks)
        elif week_for_poll > max(available_poll_weeks):
            poll_week = max(available_poll_weeks)
        else:
            poll_week = max(w for w in available_poll_weeks if w <= week_for_poll)

        standings = get_full_top25_standings_table(
            newsletter_week=poll_week,
            newsletter_date=send_date_str,
            merged_df=all_games_df,
            rankings_df=rankings_df,
            Team_Colors=Team_Colors
        )
        standings_html = build_standings_table(
            newsletter_type, games_yesterday_sorted, all_games_df, send_date_str, Team_Colors, standings_override=standings
        )
    else:
        standings_html = build_standings_table(
            newsletter_type, games_yesterday_sorted, all_games_df, send_date_str, Team_Colors
        )

    report_html += f"""
    <div style="border:2px solid #003366; border-radius:10px; background:#f9f9f9; max-width:775px; margin: 0 auto 18px auto; overflow:hidden;">
      {standings_html}
    </div>
    """

    # --- FOOTER ---
    if newsletter_type == "Top 25":
        footer_html = build_email_footer(
            games_yesterday_sorted, all_games_df, newsletter_type,
            rankings_df=rankings_df, poll_week=poll_week
        )
    else:
        footer_html = build_email_footer(
            games_yesterday_sorted, all_games_df, newsletter_type
        )
    report_html += footer_html

    final_html = f"""
    <table width="775" align="center" cellpadding="0" cellspacing="0" border="0" 
      style="background:#e8f4f8; border:4px solid #d0d0d0; border-radius:18px; box-shadow:0 8px 40px #d8d8e8; margin: 30px auto;">
      <tr>
        <td style="padding: 28px 10px 24px 10px;">
          {report_html}
        </td>
      </tr>
    </table>
    """

    final_html = minify_html(final_html)
    print(f"[{newsletter_type}] Email HTML size: {len(final_html.encode('utf-8'))} bytes")
    
    subject = f"{newsletter_type} Football Daily Recap for {send_date_str}"
    try:
        send_email_with_logos(
            subject=subject,
            html_body=final_html,
            logo_files=logo_files,
            recipients=RECIPIENTS,
            sender=SENDER,
            smtp_server=SMTP_SERVER,
            smtp_port=SMTP_PORT,
            smtp_user=SMTP_USER,
            smtp_password=SMTP_PASSWORD
        )
        print(f"Sent {subject} to {', '.join(RECIPIENTS)}")
        return True
    except Exception as e:
        print(f"Failed to send {subject}: {e}")
        return False


In [38]:
VALID_NEWSLETTERS = ['Big Ten', 'Big 12', 'ACC', 'SEC', 'Top 25', 'Ranked GO5']
# You can use: NEWSLETTER_TYPES = ['ALL'] or ['Big Ten', 'Top 25']

if isinstance(NEWSLETTER_TYPES, str) and NEWSLETTER_TYPES.upper() == "ALL":
    newsletter_list = VALID_NEWSLETTERS
elif isinstance(NEWSLETTER_TYPES, list) and "ALL" in [t.upper() for t in NEWSLETTER_TYPES]:
    newsletter_list = VALID_NEWSLETTERS
else:
    newsletter_list = NEWSLETTER_TYPES

for ntype in newsletter_list:
    if ntype == "Top 25":
        assemble_and_send_email(ntype, df_all, yesterday_str, rankings_df=rankings_df)
    else:
        assemble_and_send_email(ntype, df_all, yesterday_str)


✅ All boxscore files committed and pushed in one batch.
[Big Ten] Email HTML size: 36301 bytes
Email sent successfully.
Sent Big Ten Football Daily Recap for 2024-11-16 to will.semmer@gmail.com
